In [1]:
#import python libraries
import pandas as pd
import numpy as np

import sys
import math
import csv
import urllib
import collections

#import Scikit-Learn libraries for implementation of machine learning algorithms 
import sklearn
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Recursive Feature Elimination
from sklearn import datasets
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
#feature importance
from sklearn import metrics
from sklearn.ensemble import ExtraTreesClassifier
#to create predictions
from sklearn.model_selection import train_test_split
#algorithms tested
from sklearn import svm
from sklearn.svm import SVC
from sklearn.svm import LinearSVC
from sklearn import linear_model
from sklearn import tree
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import classification_report

### Data Visualization

In [4]:
EPL_data = pd.read_csv('premier-league-dataset/all_season_till_1718.csv')
EPL_data.head()

,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,Referee,...,AF,HC,AC,HY,AY,HR,AR,Year,Winner,Loser
0,2009-08-15,Aston Villa,Wigan,0,2,A,0,1,A,M Clattenburg,...,14,4,6,2,2,0,0,2009,Wigan,Aston Villa
1,2009-08-15,Blackburn,Man City,0,2,A,0,1,A,M Dean,...,9,5,4,2,1,0,0,2009,Man City,Blackburn
2,2009-08-15,Bolton,Sunderland,0,1,A,0,1,A,A Marriner,...,10,4,7,2,1,0,0,2009,Sunderland,Bolton
3,2009-08-15,Chelsea,Hull,2,1,H,1,1,D,A Wiley,...,15,12,4,1,2,0,0,2009,Chelsea,Hull
4,2009-08-15,Everton,Arsenal,1,6,A,0,3,A,M Halsey,...,13,4,9,0,0,0,0,2009,Arsenal,Everton


In [5]:
team_names = pd.read_csv('premier-league-dataset/Team_names.csv')
teamList = team_names['Team_Name'].tolist()
team_names.tail()

,Team_Name
11,Watford
12,Newcastle
13,Tottenham
14,Liverpool
15,Bournemouth


In [6]:
#test
print (team_names[team_names['Team_Name'] == 'Arsenal'])

  Team_Name
0   Arsenal


In [7]:
#test
print (team_names[team_names['Team_Name'] == 'Liverpool'])

    Team_Name
14  Liverpool


In [8]:
#get annual vectors for each team
def getAnnualTeamData(teamName, year):
    
    annual_data = EPL_data[EPL_data['Year'] == year]
    
    # num goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamName] 
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    # total goals allowed
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome['HY'].sum()
    totalYellowCards += gamesAway['AY'].sum()
    totalRedCards = gamesHome['HR'].sum()
    totalRedCards += gamesAway['AR'].sum()
    
    #total fouls
    totalFouls = gamesHome['HF'].sum()
    totalFouls += gamesAway['AF'].sum()
    
    #total Corners
    totalCorners = gamesHome['HC'].sum()
    totalCorners += gamesAway['AC'].sum()

    #shots per game (spg) = total shots / total games 
    totalShots = gamesHome['HS'].sum()
    # avg shots per game
    totalShots += gamesAway['AS'].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome['AS'].sum()
    totalShotsAgainst += gamesAway['HS'].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
    
    #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    totalShotsOnGoal = gamesHome['HST'].sum()
    totalShotsOnGoal += gamesAway['AST'].sum()
    goalieSaves = totalShotsOnGoal - totalGoalsAllowed
    
        #Saves Percentage = Goalie Saves / Shots on Goal   
    if totalShotsOnGoal != 0:
        savesPercentage = goalieSaves / totalShotsOnGoal
        
        #Saves Ratio = Shots On Goal / Goalie Saves    
    if goalieSaves != 0:
        savesRatio = totalShotsOnGoal / goalieSaves

    #Offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored ) / Scoring Attempts
    if totalShots != 0:
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        
        #Scoring Ratio = Shots On Goal / Goals Scored
    if totalGoalsScored != 0:
        scoringRatio = totalShotsOnGoal / totalGoalsScored       
        
            
    if numGames == 0: #if team not in dataset
        gamesWon = 0
        gamesLost = 0
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        gamesWonPercentage = 0
        goalieSaves = 0
        savesPercentage = 0
        savesRatio = 0
        scoringPercentage = 0
        scoringRatio = 0 
        
    return [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards,
        totalFouls,totalCorners, spg, sag, gamesWonPercentage, goalieSaves, savesPercentage, savesRatio,
        scoringPercentage, scoringRatio]

In [9]:
#test
list(map(int,getAnnualTeamData('Arsenal', 2017)))

[76, 51, 69, 2, 438, 266, 16, 11, 0, 191, 0, 1, 0, 3]

In [10]:
#test
getAnnualTeamData('Chelsea', 2015)

[55,
 47,
 71,
 5,
 398,
 215,
 13.736842105263158,
 11.842105263157896,
 0.4473684210526316,
 129,
 0.7329545454545454,
 1.3643410852713178,
 0.8946360153256705,
 3.2]

In [11]:
#create a dictionary for all the team stats in a year for all the teams
def createAnnualDict(year):
    annualDictionary = collections.defaultdict(list)
    for team in teamList:
        team_vector = list(map(int,getAnnualTeamData(team, year)))
        annualDictionary[team] = team_vector
    return annualDictionary

# Model Training
Create training method that takes in a dictionary with with all the teams vectors by year. For each game, the function calculates the difference between between the team vectors for that year. Then, the function assigns a yTrain that is a 1 if the home team wins, and 0 otherwise. The difference vector becomes the input (xTrain) for the model, and a label (yTrain).

In [12]:
createAnnualDict(2016)

defaultdict(list,
            {'Arsenal': [71, 37, 51, 3, 369, 204, 14, 11, 0, 153, 0, 1, 0, 2],
             'Burnley': [21, 29, 34, 0, 211, 76, 9, 18, 0, 29, 0, 2, 0, 2],
             'Everton': [49, 50, 55, 4, 381, 203, 12, 13, 0, 126, 0, 1, 0, 3],
             'Leicester': [55,
              42,
              58,
              4,
              436,
              209,
              12,
              14,
              0,
              111,
              0,
              1,
              0,
              2],
             'Man City': [73, 42, 61, 3, 415, 255, 15, 8, 0, 151, 0, 1, 0, 2],
             'Southampton': [52,
              40,
              55,
              6,
              433,
              212,
              13,
              11,
              0,
              129,
              0,
              1,
              0,
              3],
             'Swansea': [47, 72, 59, 1, 431, 185, 11, 15, 0, 75, 0, 1, 0, 3],
             'West Ham': [60, 61, 73, 3, 413, 207, 15, 13, 0, 1

In [13]:
def getTrainingData(years):
    totalNumGames = 0
    for year in years:
        annual = EPL_data[EPL_data['Year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData('Arsenal',2015)) #random team, to find dimensionality
    xTrain = np.zeros(( totalNumGames, numFeatures))
    yTrain = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict(year)
        annual = EPL_data[EPL_data['Year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                yTrainAnnual[counter] = 1
            # the opposite of the difference of the vectors should be a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain, yTrain

In [14]:
#get the dictionary
years = range(2005,2018)
xTrain, yTrain = getTrainingData(years)

In [15]:
xTrain.shape

(3249, 14)

In [16]:
yTrain.shape

(3249,)

## Feature Selection

Feature selection of the 14 features is done through recursive feature elimination and a ranking of feature importance with extra trees classifier. 10 features of the 14 were determined to be much more influential. 

In [17]:
model1 = LogisticRegression() #recursive feature elimination 
model2 = ExtraTreesClassifier() #feature importance

In [18]:
# create the RFE model and select 3 attributes
rfe = RFE(model1, 9)
rfe = rfe.fit(xTrain, yTrain)
# summarize the selection of the attributes
print(rfe.support_)
print(rfe.ranking_)

[ True  True  True  True False False  True  True False  True False  True
 False  True]
[1 1 1 1 3 2 1 1 6 1 5 1 4 1]


In [19]:
# Feature Importance
#Top Features: total goals scored, total goals allowed, total yellow cards/red cards, total fouls, total corners,
    #spg, sag, goalie saves, scoring ratio
    
#Lowest features: scoring percentage, saves percentage, games won percentage, save ratio
    
#fit an Extra Trees model to the data
model2.fit(xTrain, yTrain)
#display the relative importance of each attribute
features = ["totalGoalsScored", "totalGoalsAllowed", "totalYellowCards", "totalRedCards", "totalFouls","totalCorners", "spg", "sag", "gamesWonPercentage", "goalieSaves", "savesPercentage", "savesRatio", "scoringPercentage", "scoringRatio"]
importances = model2.feature_importances_
importance_dict = dict(zip(features,importances))
importance_dict

{'totalGoalsScored': 0.10841778138402987,
 'totalGoalsAllowed': 0.10964416579643574,
 'totalYellowCards': 0.1120935043034216,
 'totalRedCards': 0.10339743432793687,
 'totalFouls': 0.0970030693402022,
 'totalCorners': 0.10634829483188943,
 'spg': 0.09536642574915126,
 'sag': 0.09190452543431875,
 'gamesWonPercentage': 0.0,
 'goalieSaves': 0.10543869472949625,
 'savesPercentage': 0.0,
 'savesRatio': 0.01197047208662431,
 'scoringPercentage': 0.0,
 'scoringRatio': 0.05841563201649369}

### Updated Functions to include only top 10 features

In [20]:
#updated function to include only top 10 features
def getAnnualTeamData2(teamName, year):
    
    annual_data = EPL_data[EPL_data['Year'] == year]
    
    # num goals scored in wins and losses
    gamesHome = annual_data[annual_data['HomeTeam'] == teamName] 
    totalGoalsScored = gamesHome['FTHG'].sum()
    gamesAway = annual_data[annual_data['AwayTeam'] == teamName]
    totalGames = gamesHome.append(gamesAway)
    numGames = len(totalGames.index)
    #total goals scored
    totalGoalsScored += gamesAway['FTAG'].sum()
    # total goals allowed
    totalGoalsAllowed = gamesHome['FTAG'].sum()
    totalGoalsAllowed += gamesAway['FTHG'].sum()
    
    #discipline: total red cards, total yellow cards
    totalYellowCards = gamesHome['HY'].sum()
    totalYellowCards += gamesAway['AY'].sum()
    totalRedCards = gamesHome['HR'].sum()
    totalRedCards += gamesAway['AR'].sum()
    
    #total fouls
    totalFouls = gamesHome['HF'].sum()
    totalFouls += gamesAway['AF'].sum()
    
    #total Corners
    totalCorners = gamesHome['HC'].sum()
    totalCorners += gamesAway['AC'].sum()

    #shots per game (spg) = total shots / total games 
    totalShots = gamesHome['HS'].sum()
    # avg shots per game
    totalShots += gamesAway['AS'].sum()
    if numGames != 0:
        spg = totalShots / numGames
    # avg shots allowed per game
    totalShotsAgainst = gamesHome['AS'].sum()
    totalShotsAgainst += gamesAway['HS'].sum()
    if numGames != 0:
        sag = totalShotsAgainst / numGames
    
    #Games Won Percentage = Games Won / (Games Won + Games Lost) 
    gamesWon = annual_data[annual_data['Winner'] == teamName] 
    gamesLost = annual_data[annual_data['Loser'] == teamName] 
    numGamesWon = len(gamesWon.index)
    numGamesLost = len(gamesLost.index)
    if numGames != 0:
        gamesWonPercentage = numGamesWon / numGames
    
    #Defense stats
        #Goalie Saves = Shots on Goal - Goal Scored
    totalShotsOnGoal = gamesHome['HST'].sum()
    totalShotsOnGoal += gamesAway['AST'].sum()
    goalieSaves = totalShotsOnGoal - totalGoalsAllowed
    
        #Saves Percentage = Goalie Saves / Shots on Goal   
    if totalShotsOnGoal != 0:
        savesPercentage = goalieSaves / totalShotsOnGoal
        
        #Saves Ratio = Shots On Goal / Goalie Saves    
    if goalieSaves != 0:
        savesRatio = totalShotsOnGoal / goalieSaves

    #Offense stats
        #Scoring Percentage = (Scoring Attempts - Goals Scored ) / Scoring Attempts
    if totalShots != 0:
        scoringPercentage = (totalShots - totalGoalsScored) / totalShots
        
        #Scoring Ratio = Shots On Goal / Goals Scored
    if totalGoalsScored != 0:
        scoringRatio = totalShotsOnGoal / totalGoalsScored       
        
            
    if numGames == 0: #team not in dataset
        totalGoalsScored = 0
        totalGoalsAllowed = 0
        totalYellowCards = 0
        totalRedCards = 0
        totalFouls = 0
        totalCorners = 0
        spg = 0
        sag = 0
        goalieSaves = 0
        scoringRatio = 0
        
    return [totalGoalsScored, totalGoalsAllowed, totalYellowCards, totalRedCards,
        totalFouls,totalCorners, spg, sag, goalieSaves, scoringRatio]

In [21]:
#test
getAnnualTeamData2('Chelsea', 2015)

[55, 47, 71, 5, 398, 215, 13.736842105263158, 11.842105263157896, 129, 3.2]

In [22]:
#updated functions to include only top 10 features
def createAnnualDict2(year):
    annualDictionary = collections.defaultdict(list)
    for team in teamList:
        team_vector = list(map(int,getAnnualTeamData2(team, year)))
        annualDictionary[team] = team_vector
    return annualDictionary

In [23]:
createAnnualDict2(2016)

defaultdict(list,
            {'Arsenal': [71, 37, 51, 3, 369, 204, 14, 11, 153, 2],
             'Burnley': [21, 29, 34, 0, 211, 76, 9, 18, 29, 2],
             'Everton': [49, 50, 55, 4, 381, 203, 12, 13, 126, 3],
             'Leicester': [55, 42, 58, 4, 436, 209, 12, 14, 111, 2],
             'Man City': [73, 42, 61, 3, 415, 255, 15, 8, 151, 2],
             'Southampton': [52, 40, 55, 6, 433, 212, 13, 11, 129, 3],
             'Swansea': [47, 72, 59, 1, 431, 185, 11, 15, 75, 3],
             'West Ham': [60, 61, 73, 3, 413, 207, 15, 13, 106, 2],
             'Crystal Palace': [45, 68, 72, 0, 459, 208, 12, 14, 75, 3],
             'Man United': [56, 38, 77, 2, 486, 237, 14, 11, 150, 3],
             'West Brom': [41, 47, 68, 0, 429, 167, 10, 13, 61, 2],
             'Watford': [38, 60, 80, 4, 495, 160, 11, 13, 75, 3],
             'Newcastle': [25, 31, 23, 2, 215, 77, 11, 12, 61, 3],
             'Tottenham': [69, 33, 66, 0, 445, 275, 19, 10, 212, 3],
             'Liverpool': [87,

In [24]:
#updated functions to include only top 10 features
def getTrainingData2(years):
    totalNumGames = 0
    for year in years:
        annual = EPL_data[EPL_data['Year'] == year]
        totalNumGames += len(annual.index)
    numFeatures = len(getAnnualTeamData2('Arsenal',2015)) #random team, to find dimensionality
    xTrain2 = np.zeros(( totalNumGames, numFeatures))
    yTrain2 = np.zeros(( totalNumGames ))
    indexCounter = 0
    for year in years:
        team_vectors = createAnnualDict2(year)
        annual = EPL_data[EPL_data['Year'] == year]
        numGamesInYear = len(annual.index)
        xTrainAnnual = np.zeros(( numGamesInYear, numFeatures))
        yTrainAnnual = np.zeros(( numGamesInYear ))
        counter = 0
        for index, row in annual.iterrows():
            h_team = row['HomeTeam']
            h_vector = team_vectors[h_team]
            a_team = row['AwayTeam']
            a_vector = team_vectors[a_team]
            diff = [a - b for a, b in zip(h_vector, a_vector)]
            if (counter % 2 == 0):
                if len(diff) != 0:
                    xTrainAnnual[counter] = diff
                if h_team == row['Winner']:
                    yTrainAnnual[counter] = 1
                else: 
                    yTrainAnnual[counter] = 0
            # the opposite of the difference of the vectors should be a true negative, where team 1 does not win
            else:
                if len(diff) != 0:
                    xTrainAnnual[counter] = [ -p for p in diff]
                yTrainAnnual[counter] = 0
            counter += 1
        xTrain2[indexCounter:numGamesInYear+indexCounter] = xTrainAnnual
        yTrain2[indexCounter:numGamesInYear+indexCounter] = yTrainAnnual
        indexCounter += numGamesInYear
    return xTrain2, yTrain2

In [25]:
#get the dictionary
years = range(2005,2018)
xTrain2, yTrain2 = getTrainingData2(years)

In [26]:
xTrain2.shape

(3249, 10)

In [27]:
yTrain2.shape

(3249,)

In [28]:
print(xTrain2)

[[   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 [   0.    0.    0. ...    0.    0.    0.]
 ...
 [  11.   17.   21. ...    0.    7.   -1.]
 [  63.  -32.   -8. ...   -6.  154.   -1.]
 [ -43.    5.   10. ...    2. -119.    0.]]


## Model Validation

In [29]:
# Tried all the following models. Uncomment model to try.
lms = [
    linear_model.LinearRegression(),
    tree.DecisionTreeClassifier(),
    tree.DecisionTreeRegressor(),
    linear_model.LogisticRegression(),
    linear_model.BayesianRidge(),
    linear_model.Lasso(),
    svm.SVC(),
    svm.SVR(),
    linear_model.Ridge(alpha = 0.5),
    AdaBoostClassifier(n_estimators=100),
    GradientBoostingClassifier(n_estimators=100),
    GradientBoostingRegressor(n_estimators=100, max_depth=9),
    RandomForestClassifier(n_estimators=100)
]

In [30]:
xTrain, X_test, yTrain, y_test = train_test_split(xTrain, yTrain,test_size=0.3)
print(xTrain.shape, yTrain.shape)
print(X_test.shape, y_test.shape)

(2274, 14) (2274,)
(975, 14) (975,)


In [31]:
for lm in lms:
    lm.fit(xTrain, yTrain)
    predictions = lm.predict(X_test)
    #avg pred
    print(lm.__class__.__name__,sum(predictions)/len(predictions))
    #print predictions

LinearRegression 0.5016731055558938
DecisionTreeClassifier 0.16512820512820514
DecisionTreeRegressor 0.49323048551264687
LogisticRegression 0.8184615384615385
BayesianRidge 0.5014493963356438
Lasso 0.5013192612137229
SVC 0.3087179487179487
SVR 0.24729051739185792
Ridge 0.5016738888427577
AdaBoostClassifier 0.8574358974358974
GradientBoostingClassifier 0.19589743589743588
GradientBoostingRegressor 0.5083371676269942
RandomForestClassifier 0.18769230769230769


## Results
Calculate the predictions for the 2019 games using 2018 data. 

In [32]:
def createGamePrediction(team1_vector, team2_vector, xTrain, yTrain):
    xTrain, X_test, yTrain, Y_test = train_test_split(xTrain, yTrain)
    xTrain.shape, yTrain.shape
    X_test.shape, y_test.shape
    lm = linear_model.LinearRegression()
    model2 = lm.fit(xTrain, yTrain)
    diff = np.reshape([a - b for a, b in zip(team1_vector, team2_vector)],(1,-1))
    predictions = lm.predict(diff)
    return predictions[0]

In [33]:
team1_name = 'Arsenal'
team2_name = 'West Brom'
team1_vector = getAnnualTeamData(team1_name, 2018)
team2_vector = getAnnualTeamData(team2_name, 2018)
team3_vector = getAnnualTeamData("Chelsea", 2018)

print ('Probability that ' + team1_name + ' wins:',createGamePrediction(team1_vector, team2_vector,xTrain, yTrain))
print ('Probability that ' + team2_name + ' wins:',createGamePrediction(team2_vector, team1_vector,xTrain, yTrain))

Probability that Arsenal wins: 0.5538881025440098
Probability that West Brom wins: 0.4507117516213019


In [35]:
team1_vector = getAnnualTeamData("Arsenal", 2017)
team2_vector = getAnnualTeamData("Crystal Palace", 2017)
print(createGamePrediction(team1_vector, team2_vector,xTrain, yTrain))

0.5297111415044805


In [36]:
team1_vector = getAnnualTeamData("Burnley", 2017)
team2_vector = getAnnualTeamData("Man United", 2017)
print(createGamePrediction(team1_vector, team2_vector,xTrain, yTrain))

0.47860113833701967


In [37]:
#game_ID given to each game to simplify identification of game
def formulatePredictions():
    probs = [[0 for x in range(2)] for x in range(len(test_data.index))]
    for index, row in test_data.iterrows():
        game_ID = row['Game_ID']
        year = row['Year'] - 1
        team1_Name = row['HomeTeam']
        team2_Name = row['AwayTeam']
        team1_vector = getAnnualTeamData(team1_Name, year)
        team2_vector = getAnnualTeamData(team2_Name, year)
        prediction = createGamePrediction(team1_vector, team2_vector,xTrain, yTrain)
        probs[index][0] = game_ID
        probs[index][1] = prediction
    probs = pd.np.array(probs)
    return probs